<a href="https://colab.research.google.com/github/glorivaas/Risk_Measures/blob/main/lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 1

### Example risk measures (L/P)
- VaR (value at risk) at the confidence level $c\in(0,1)$
$$ \operatorname{VaR}_c(L) = \inf\{\, v\in\mathbb{R} \colon \mathbb{P}(L > v) \le 1-c \,\} $$
- ES (expected shortfall) at the confidence level $c\in(0,1)$
$$ \operatorname{ES}_c(L) = \mathbb{E}[L \,\vert\, L \ge \operatorname{VaR}_c(L)] $$
- CVaR (conditional VaR) at the confidence level $c\in(0,1)$
$$ \operatorname{CVaR}_c(L) = \frac{1}{1-c} \int_c^1 \operatorname{VaR}_\xi(L) \,d\xi $$

### Exercise 0
Setup your python environment, install numpy, scipy, matplotlib (or other visualization libraries depending on your preferences).

### Exercise 1
Complete the implementation of the following functions.

In [ ]:
Complete the implementation of the following functions.

### Exercise 2
Check the implementations from task 1 vs. analytical formulas for the uniform distribution on $[0,1]$, $\operatorname{Unif}([0,1])$.


In [ ]:


def test_vs_uniform() -> None:
    c = 0.95
    def cdf_unif(x: NumArr) -> NumArr:
        return np.minimum(np.maximum(x, 0.0), 1.0)

    assert np.isclose(var(cdf_unif, c), ...)
    assert np.isclose(es(cdf_unif, c), ...)
    assert np.isclose(cvar(cdf_unif, c), ...)

test_vs_uniform()


### Exercise 3
- For the standard normal distribution $\mathcal{N}(0,1)$, plot the above risk measures against the confidence level $c\in(0,1)$.
- For the normal distribution $\mathcal{N}(0,\sigma)$, plot the above risk measures against $\sigma>0$ for the confidence level $c=95\%$.

Describe the results.